In [ ]:
import os, torch, numpy as np
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
class ShipsDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths     = paths     # list of file‐paths (strings)
        self.labels    = labels    # list of ints
        self.transform = transform
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        img_path = self.paths[idx]
        image    = Image.open(img_path).convert("RGB")   # ← load PIL image
        if self.transform:
            image = self.transform(image)                # ← apply ToTensor+Normalize
        label = self.labels[idx]
        return image, label

BATCH_SIZE = 64  # or whatever you set earlier

train_loader = DataLoader(
    ShipsDataset(train_paths, train_labels, transform),
    batch_size=BATCH_SIZE, shuffle=True,  num_workers=2
)
val_loader = DataLoader(
    ShipsDataset(val_paths, val_labels, transform),
    batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)
test_loader = DataLoader(
    ShipsDataset(test_paths, test_labels, transform),
    batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

print("Rebuilt DataLoaders:", len(train_loader), len(val_loader), len(test_loader))


In [ ]:
#Build frozen ResNet‑18 feature extractor
base_model = models.resnet18(pretrained=True)
feature_extractor = nn.Sequential(*list(base_model.children())[:-1]).to(device)
feature_extractor.eval()


NameError: name 'device' is not defined

In [ ]:
@torch.no_grad()
def extract_features(loader):
    feats, labs = [], []
    for imgs, labels in loader:
        imgs = imgs.to(device)
        out  = feature_extractor(imgs)        # → [B,512,1,1]
        out  = out.view(out.size(0), -1)      # → [B,512]
        feats.append(out.cpu())
        labs.append(torch.tensor(labels))
    return torch.cat(feats, dim=0), torch.cat(labs, dim=0)

train_feats, train_labels = extract_features(train_loader)
test_feats,  test_labels  = extract_features(test_loader)
print(f"Train feats: {train_feats.shape}, Train labels: {train_labels.shape}")
print(f"Test  feats: {test_feats.shape},  Test labels: {test_labels.shape}")


In [ ]:
# Cell 9: Pure‑PyTorch KNN and evaluation
def knn_predict(test_feats, train_feats, train_labels, k=5):
    dists    = torch.cdist(test_feats, train_feats)  # [N_test,N_train]
    idxs     = dists.topk(k, largest=False).indices  # [N_test,k]
    knn_labs = train_labels[idxs]                    # [N_test,k]
    preds    = []
    for row in knn_labs:
        vals, counts = row.unique(return_counts=True)
        preds.append(vals[counts.argmax()].item())
    return torch.tensor(preds)

k     = 5
preds = knn_predict(test_feats, train_feats, train_labels, k)
acc   = (preds == test_labels).float().mean().item()
print(f"{k}-NN test accuracy: {acc * 100:.2f}%")
